## Model description

This benchmark consists of a reactor tank and two independent rods. The goal is to control the coolant temperature $x$ in the tank by putting in or taking out the rods [1].

The rods differ in their cooling dynamics. Two clocks, $c_1$ and $c_2$, are introduced to measure the time elapsed since the last use
of respectively `rod1` and `rod2`.

This model has 3 continuous variables, 2 modes and 2 discrete jumps.

The syste can be represented as the following hybrid automaton:

<img src="RodReactor.png" alt="Drawing" style="width: 400px;"/>

The model parameters and description are taken from the [HyPro collection of continuous and hybrid system benchmarks](https://ths.rwth-aachen.de/research/projects/hypro/benchmarks-of-continuous-and-hybrid-systems/), see the [Rod Reactor model](https://ths.rwth-aachen.de/research/projects/hypro/rod-reactor/).

---

*References:*

[1] F. Vaandrager. Lecture notes: Hybrid Systems.

In [1]:
using Revise # to debug
using Reachability, HybridSystems, MathematicalSystems, LazySets, LinearAlgebra
using Plots, LaTeXStrings

┌ Warning: Package LazySets does not have Expokit in its dependencies:
│ - If you have LazySets checked out for development and have
│   added Expokit as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with LazySets
│ Loading Expokit into LazySets from project dependency, future warnings for LazySets are suppressed.
└ @ nothing nothing:840


In [26]:
function rod_reactor(;X0=Singleton([510.0, 20.0, 20.0]),
                      T=5.0,
                      ε=1e-6)

    # automaton structure
    automaton = LightAutomaton(4)

    # rod_1  : x' = Ax + b
    A = [0.1 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
    b = [-56.0, 1.0, 1.0]
    X = HPolyhedron([HalfSpace([-1.0, 0.0, 0.0], -510.0)]) # x >= 510
    rod_1 = ConstrainedAffineContinuousSystem(A, b, X)
    
    # no_rods
    b = [-50.0, 1.0, 1.0]
    X = HPolyhedron([HalfSpace([1.0, 0.0, 0.0], 550.0)])   # x <= 550
    no_rods = ConstrainedAffineContinuousSystem(A, b, X)

    # rod_2
    b = [-60.0, 1.0, 1.0]
    X = HPolyhedron([HalfSpace([-1.0, 0.0, 0.0], -510.0)])   # x >= 510
    rod_2 = ConstrainedAffineContinuousSystem(A, b, X)

    # shut_down
    X = Universe(2)
    shut_down = ConstrainedContinuousIdentitySystem(3, X)

    # modes
    modes = [rod_1, no_rods, rod_2, shut_down]

    add_transition!(automaton, 1, 2, 1)
    add_transition!(automaton, 2, 1, 2)
    add_transition!(automaton, 2, 3, 3)
    add_transition!(automaton, 3, 2, 4)
    add_transition!(automaton, 2, 4, 5)

    # guards
    G12 = HPolyhedron([HalfSpace([1.0, 0.0, 0.0], 510.0 + ε),
                       HalfSpace([-1.0, 0.0, 0.0], -510.0 + ε)])  # x = 510

    G21 = HPolyhedron([HalfSpace([1.0, 0.0, 0.0], 550.0 + ε),
                       HalfSpace([-1.0, 0.0, 0.0], -550.0 + ε),   # x = 550
                       HalfSpace([0.0, -1.0, 0.0], -20.0)])       # c1 >= 20

    G23 = HPolyhedron([HalfSpace([1.0, 0.0, 0.0], 550.0 + ε),
                       HalfSpace([-1.0, 0.0, 0.0], -550.0 + ε),   # x = 550
                       HalfSpace([0.0, 0.0, -1.0], -20.0)])       # c2 >= 20

    G32 = HPolyhedron([HalfSpace([1.0, 0.0, 0.0], 510.0 + ε),
                       HalfSpace([-1.0, 0.0, 0.0], -510.0 + ε)])  # x = 510

    G24 = HPolyhedron([HalfSpace([1.0, 0.0, 0.0], 550.0 + ε),
                       HalfSpace([-1.0, 0.0, 0.0], -550.0 + ε),  # x = 550
                       HalfSpace([1.0, 0.0, 0.0], 20.0),         # c1 < 20
                       HalfSpace([0.0, 1.0, 0.0], 20.0)])        # c2 < 20
    
    resetmaps = [ConstrainedResetMap(2, G12, Dict(2=>0.0)), 
                 ConstrainedIdentityMap(2, G21),
                 ConstrainedIdentityMap(2, G23),
                 ConstrainedResetMap(2, G32, Dict(3=>0.0)),
                 ConstrainedIdentityMap(2, G24)]

    # switching
    switchings = [AutonomousSwitching()]

    ℋ = HybridSystem(automaton, modes, resetmaps, switchings)

    # initial condition in "off_off" mode
    initial_condition = [(1, X0)]

    problem = InitialValueProblem(ℋ, initial_condition)

    options = Options(:mode=>"reach", :T=>T, :plot_vars=>[1, 3],
                      :project_reachset=>false)

    return (problem, options)
end

rod_reactor (generic function with 3 methods)

## Reachability settings

We consider an initial set of

$$
x = 510,~c_1 = c_2 = 20.
$$
The initial location is the location "No Rods" and the time horizon $T= 50s$.

The set of unreachable states are all states in location shut-down.

In [27]:
# settings
X0 = Singleton([510.0, 20.0, 20.0])
T = 5.0

RodReactor, options = rod_reactor(X0=X0, T=T);

## Results

In [30]:
using Polyhedra

In [32]:
@time begin
    opC = BFFPSV18(:δ=>0.1)
    opD = LazyDiscretePost()
    sol = solve(RodReactor, options, opC, opD)
end;

ErrorException: the exact support vector of an intersection is not implemented

In [33]:
plot(sol)

UndefVarError: UndefVarError: sol not defined

In [12]:
using Polyhedra, CDDLib

In [13]:
@time begin
    sol = solve(TwoTank, options,
                BFFPSV18(:δ=>0.001),
                ConcreteDiscretePost())
end;

  0.035339 seconds (2.75 k allocations: 597.000 KiB)


In [ ]:
plot(sol, xlab=L"x_1", ylab=L"x_2")